# Find the nearest point to another set of points

#### Workflow
1. Import packages
2. Load shapefile1 (the original points) and shapefile2 (the points whose values to bring into shapefile1)
3. Run function ckdnearest, given shp1, shp2, and the column from shp2 that is of interest
4. Include the bcol into shp1 by copying it from the new df (nearest)
5. (Optional) convert the distanc to feet from meters

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.interpolate import griddata
from shapely.ops import nearest_points
from scipy.spatial import cKDTree  
from shapely.geometry import Point, LineString
import shapely.geometry as geom

In [ ]:
shp1 = gpd.read_file("shp1.shp") # shp 1
shp2 = gpd.read_file("shp2.shp") # shp 2

In [ ]:
def ckdnearest(gdA, gdB, bcol):   
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA)
    df = pd.DataFrame.from_dict({'Dist(meters)': dist.astype(int),
                             'Z' : gdB.loc[idx, bcol].values })
    return df

In [ ]:
nearest = ckdnearest(gdA=shp1, gdB=shp2, bcol='Z')

In [ ]:
shp1["Z"] = nearest["Z"]
shp1["Dist(ft)"] = nearest["Dist(meters)"]*3.28084 # convert to feet

In [ ]:
shp1.to_file("path\\to\\file.shp")